In [1]:
%load_ext cython
%matplotlib inline
import sys
sys.path.append('/home/bram/ANTS')
sys.path.append('/home/bram/ANTS/entropy')
for p in sys.path:
    print(p)


/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload
/home/bram/PythonEnvs/Ants/lib/python3.6/site-packages
/home/bram/PythonEnvs/Ants/lib/python3.6/site-packages/IPython/extensions
/home/bram/.ipython
/home/bram/ANTS
/home/bram/ANTS/entropy


In [3]:
%%cython -a
# distutils: language = c++

import numpy as np

from cythonic.core.queen cimport Queen
from cythonic.core.ant cimport Ant, ant_state
from cythonic.core.domain cimport Domain
from cythonic.plugins.dummy_dicts import ant_dict,domain_dict, queen_dict
from cythonic.plugins.positions cimport point


import matplotlib.pyplot as plt

from cythonic.plugins.sens_structs cimport  observations
cdef void check(point * p, observations * q):
    print(p[0])
    print(q[0])


    
def new_positions(deploy_style, n, **kwargs):
    xy = np.zeros([n,2], dtype = np.float)
    tetas = np.zeros(n,dtype=np.float)
    
    if deploy_style =='nest_radian':
        " all ants start aligned with a radian originating from the nest"
        x = kwargs['nest_loc']['x']
        y = kwargs['nest_loc']['y']
        R = kwargs['R']
        for i in range(n): 
            # expect nest location [nest_loc] and radius [R] in kwargs
            teta = np.random.rand()*2*np.pi
            xy[i,0] = x+np.cos(teta)*R
            xy[i,1] = y+np.sin(teta)*R
            tetas[i] = teta*180/np.pi
            
    return xy, tetas
            
def deploy_times(deploy_method, n, **kwargs):
    " generate a list of time instances when an ant is to be deployed "
    if deploy_method == 'instant':
        " deploy all at time t==0 "
        return np.zeros(n,dtype = np.float)
    elif deploy_method == 'uniform_dist':
        " deploy times uniformly distributed between 0 and kwargs['t_max']"
        return np.sort(kwargs['t_max']*np.random.rand(n))
    elif deploy_method == 'gamma_dist':
        """ deploy times gamma distribution 
        (https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.gamma.html)
        shape k = kwargs['k'], scale theta (inverse of 'steepness' of the pdf) = kwargs['teta'] 
        median at aprox k*theta distr"""
        return np.sort(np.random.gamma(size = n,shape = kwargs['k'],scale=kwargs['teta'] ))
    
    
cdef class Sim:
    cdef Queen queen
    cdef Domain domain
    
    cdef:
        readonly bint deployed #flag: all ants active == True
        readonly double[:] deploy_times #list of timers when ant should become active
        readonly double dt #simulation time step
        readonly double t #simulation time
    
    def __cinit__(self, queen_args, domain_args, double dt, deploy_style, ):
        
        " bring the controller online "
        self.queen = Queen(**queen_args)
        
        " initialize the domain"
        self.domain = Domain(**domain_args)
        
        " set simulation variables "
        self.t = 0.
        self.dt = dt
        tmp = np.zeros(50,dtype = np.float).tolist()
        tmp2 = tmp+np.sort(np.random.rand(1,50)[0,:]).tolist()
        self.deploy_times = np.array(tmp2)
        
        " initialize the ant states "
        xy, tetas = new_positions(deploy_style, self.queen.n,**{'R':self.domain.nest_radius, 'nest_loc':self.domain.nest_location})        
        self.queen.initialize_states(xy,tetas)
        
        "activate ants for the first time step if necessary"
        self.expand_active()
        print('Active ants: ',self.queen.count_active)
        self.t+=self.dt
        self.expand_active()
        print('Active ants: ', self.queen.count_active)
        
        self.sim_step()
        
    cdef readonly void sim_step(self):
        " do a step in the simulation, perform all simulation logic (boundary check e.g..)"
        cdef unsigned int i
        cdef observations * obs = &self.queen.pheromone_vec[1]
#         cdef point * pos = &self.queen.state_list[1].pos
        cdef point p = {'x': 1.,'y':1.}
        cdef point * pos = &p
        self.queen.pheromone_vec[1].lft = self.domain.probe_pheromone(pos)
        
        obs[0].lft = self.domain.probe_pheromone(pos)
        obs[0].rght = self.domain.probe_pheromone(pos)
        print(self.queen.)
#         print(self.queen.state_list[1])
#         for i in range(self.queen.count_active):
#             #target ant
#             self.queen.assign_state(&i)
            
            #sense (queen/domain)
            
#             self.domain.fill_observation(&self.queen.pheromone_vec[i],self.queen.agent.state.left,
#                                                                    self.queen.agent.state.right)
            
            #step (queen)
#             self.queen.agent.step
            

        #step all (queen)
        #check conditions (boundary, nest and food)
        #deposit all (domain)
        #evaporate (domain)
        
        
    cdef void expand_active(self):
        " check if new agents are to be activated based on simulation time "
        cdef unsigned int i
        for i in range(self.queen.count_active, self.queen.n,1):
            if self.deploy_times[i] <= self.t:
                " deploy ant if its deploy time is  \seq (<=) than current time step"
                self.queen.deploy(i) # deploy ant by id
            
        
        
    

#     def deploy()
    
#     cdef void parse_pheromone(self):
#         for i in range():
            # Domain.lookup(&queen.state[i].pos, &queen.pheromone_vic[i].data())

queen_dict['n'] = 100      
cdef Sim sim = Sim(queen_args = queen_dict, domain_args = domain_dict, dt = .25, deploy_style = 'nest_radian')
# print(deploy_times('instant',n=10))
# print(deploy_times('uniform_dist',n=10,t_max = 5))
# import matplotlib.pyplot as plt
# x = deploy_times('gamma_dist', n=1000, teta = 4, k = 20)
# print(f'median: {np.median(x)}, max: {np.max(x)}')
# plt.hist(x)
# plt.show()

In [5]:
%%cython -a
import numpy as np

cdef double[::,:] xy = np.array([[1.,2.],[3.,4.]])
print(xy[0,0])
print(xy[1,0])
print(np.random.rand(10))


Error compiling Cython file:
------------------------------------------------------------
...

cdef double[::,:] xy = np.array([[1.,2.],[3.,4.]])
print(xy[0,0])
print(xy[1,0])
print(np.random.rand(10))
print(deploy_times('instant',n=10))
     ^
------------------------------------------------------------

/home/bram/.cache/ipython/cython/_cython_magic_5d568b57cc52a0bcf25a5fccf1baf29a.pyx:7:6: undeclared name not builtin: deploy_times


TypeError: object of type 'NoneType' has no len()